In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel(f'./part_2.xlsx',
                       header=0,
                       dtype=str).fillna('')

print(f'总数量：{len(input_)}')
input_['JOIN_MPNTCK'].tolist()

总数量：11472


['DORMAN;600500;10775;4391568',
 'DORMAN;600502;10775;1268085',
 'DORMAN;600503;10775;1268171',
 'DORMAN;600504;10775;1268702',
 'DORMAN;600506;10775;1268353',
 'DORMAN;600550;10775;1268272',
 'DORMAN;600551;10775;1268448',
 'DORMAN;600552;10775;1268091',
 'DORMAN;600553;10775;4391583',
 'DORMAN;600554;10775;2721730',
 'DORMAN;600555;10775;8164020',
 'DORMAN;600558;10775;8164024',
 'GM GENUINE;15598481;10775;10494760',
 'GM GENUINE;15693423;10775;9667108',
 'GM GENUINE;89059420;10775;10417544',
 'ROSTRA;730010;10775;15057157',
 'ROSTRA;730014;10775;15057173',
 'SKP;SK912351;10775;11194681',
 'STANDARD MOTOR PRODUCTS;NS268;10775;444741',
 'STANDARD MOTOR PRODUCTS;TCA109;10775;12770825',
 'STANDARD MOTOR PRODUCTS;TCA11;10775;443290',
 'STANDARD MOTOR PRODUCTS;TCA25;10775;3776244',
 'STANDARD MOTOR PRODUCTS;TCA26;10775;3784134',
 'STANDARD MOTOR PRODUCTS;TCA28;10775;3787275',
 'STANDARD MOTOR PRODUCTS;TCA4;10775;320327',
 'STANDARD MOTOR PRODUCTS;TCA42;10775;5414430',
 'STANDARD MOTOR PRO

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

payload = {'partData': {'listing_data_essential': {'parttype': input_.loc[a, 'Type Code'],
                                                   'partkey': input_.loc[a, 'Key']},
                        'listing_data_supplemental': {'partnumber': input_.loc[a, 'Part Number'],
                                                      'catalogname': input_.loc[a, 'Manufacturer']}}}

data = {'func': 'getbuyersguide',
        'payload': json.dumps(payload),
        'api_json_request': '1'}

b = 0
while True:

    while True:
        b += 1
        print(b)
        try:
            resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                 data=data,
                                 headers=get_header(),
                                 proxies=get_proxy(),
                                 timeout=(10, 10))
            break
        except:
            continue

    if resp.status_code == 200:
        break

resp

1
2
3
4
5
6
7
8
9


<Response [200]>

In [6]:
json_ = resp.json()

json_

{'buyersguidepieces': {'body': '<div class="buyersguide-nested"><div style="padding: .5em;"><table class="nobmp" style="width: 100%;"><tr><td class="altrow-a-0">CHEVROLET</td><td class="altrow-a-0">BLAZER</td><td class="altrow-a-0">1995-2005</td></tr><tr><td class="altrow-a-1">CHEVROLET</td><td class="altrow-a-1">S10 BLAZER</td><td class="altrow-a-1">1983-1994</td></tr><tr><td class="altrow-a-0">CHEVROLET</td><td class="altrow-a-0">S10 PICKUP</td><td class="altrow-a-0">1983-2004</td></tr><tr><td class="altrow-a-1">GMC</td><td class="altrow-a-1">JIMMY</td><td class="altrow-a-1">1992-2001</td></tr><tr><td class="altrow-a-0">GMC</td><td class="altrow-a-0">S15 JIMMY</td><td class="altrow-a-0">1983-1991</td></tr><tr><td class="altrow-a-1">GMC</td><td class="altrow-a-1">S15 PICKUP</td><td class="altrow-a-1">1983-1990</td></tr><tr><td class="altrow-a-0">GMC</td><td class="altrow-a-0">SONOMA</td><td class="altrow-a-0">1991-2004</td></tr><tr><td class="altrow-a-1">GMC</td><td class="altrow-a-1"

In [7]:
'redirect_to_url' in json_

False

In [8]:
json_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in json_

True

In [9]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['buyersguidepieces']['body'], 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="buyersguide-nested">
   <div style="padding: .5em;">
    <table class="nobmp" style="width: 100%;">
     <tr>
      <td class="altrow-a-0">
       CHEVROLET
      </td>
      <td class="altrow-a-0">
       BLAZER
      </td>
      <td class="altrow-a-0">
       1995-2005
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       CHEVROLET
      </td>
      <td class="altrow-a-1">
       S10 BLAZER
      </td>
      <td class="altrow-a-1">
       1983-1994
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       CHEVROLET
      </td>
      <td class="altrow-a-0">
       S10 PICKUP
      </td>
      <td class="altrow-a-0">
       1983-2004
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       GMC
      </td>
      <td class="altrow-a-1">
       JIMMY
      </td>
      <td class="altrow-a-1">
       1992-2001
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       GMC
      </td>
      <td class="altrow-a-0

In [10]:
len(html.xpath('//p/text()')) != 0 and html.xpath('//p/text()')[0] == 'No applications found.'

False

In [11]:
list_tr = html.xpath('//tr')

list_tr

[<Element tr at 0x125b0ba80>,
 <Element tr at 0x125d1db80>,
 <Element tr at 0x125d1f700>,
 <Element tr at 0x125d1ccc0>,
 <Element tr at 0x125d1e980>,
 <Element tr at 0x125d1e4c0>,
 <Element tr at 0x125d1da80>,
 <Element tr at 0x125d1e200>,
 <Element tr at 0x125d1d200>,
 <Element tr at 0x125d1c740>]

In [12]:
len(list_tr) == 0

False

In [13]:
dict_vehicle = {}
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_vehicle[str(i)] = {}
    list_td = [text.strip() for text in tr.xpath('./td/text()')]
    for j, td in zip(range(len(list_td)), list_td):
        dict_vehicle[str(i)][str(j)] = td

dict_vehicle

{'0': {'0': 'CHEVROLET', '1': 'BLAZER', '2': '1995-2005'},
 '1': {'0': 'CHEVROLET', '1': 'S10 BLAZER', '2': '1983-1994'},
 '2': {'0': 'CHEVROLET', '1': 'S10 PICKUP', '2': '1983-2004'},
 '3': {'0': 'GMC', '1': 'JIMMY', '2': '1992-2001'},
 '4': {'0': 'GMC', '1': 'S15 JIMMY', '2': '1983-1991'},
 '5': {'0': 'GMC', '1': 'S15 PICKUP', '2': '1983-1990'},
 '6': {'0': 'GMC', '1': 'SONOMA', '2': '1991-2004'},
 '7': {'0': 'GMC', '1': 'SYCLONE', '2': '1991'},
 '8': {'0': 'GMC', '1': 'TYPHOON', '2': '1992-1993'},
 '9': {'0': 'OLDSMOBILE', '1': 'BRAVADA', '2': '1991-2001'}}

In [14]:
df_temp = pd.DataFrame([{'JOIN_MPNTCK': input_.loc[a, 'JOIN_MPNTCK'],
                         'Json_Vehicle': json.dumps(dict_vehicle)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,JOIN_MPNTCK,Json_Vehicle
0,DORMAN;600500;10775;4391568,"{""0"": {""0"": ""CHEVROLET"", ""1"": ""BLAZER"", ""2"": ""..."


In [15]:
crawler_status = 'ok'

crawler_status

'ok'

In [16]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'JOIN_MPNTCK': input_.loc[a, 'JOIN_MPNTCK'],
                             'Manufacturer': input_.loc[a, 'Manufacturer'],
                             'Part Number': input_.loc[a, 'Part Number'],
                             'Type Code': input_.loc[a, 'Type Code'],
                             'Key': input_.loc[a, 'Key']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [17]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['JOIN_MPNTCK'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,JOIN_MPNTCK,Json_Vehicle
0,DORMAN;600500;10775;4391568,"{""0"": {""0"": ""CHEVROLET"", ""1"": ""BLAZER"", ""2"": ""..."


In [18]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['JOIN_MPNTCK'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
